# Análisis Exploratorio de Datos (EDA)
## Proyecto de Clasificación Multiclase

Este notebook contiene el análisis exploratorio del dataset para entender:
- Distribución de las clases objetivo
- Características del dataset (dimensiones, tipos de datos)
- Valores faltantes
- Estadísticas descriptivas
- Correlaciones entre variables
- Visualizaciones clave

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Carga de Datos

In [ ]:
# Cargar el dataset
data_path = Path('../datasets/train.csv')
df = pd.read_csv(data_path)

print(f"Dimensiones del dataset: {df.shape}")
print(f"Número de registros: {df.shape[0]:,}")
print(f"Número de características: {df.shape[1]:,}")

In [ ]:
# Primeras filas del dataset
df.head(10)

In [ ]:
# Información general del dataset
df.info()

## 2. Análisis de la Variable Objetivo

In [ ]:
# Distribución de clases
print("\n=== DISTRIBUCIÓN DE CLASES ===")
print(df['target'].value_counts())
print("\nProporción de clases:")
print(df['target'].value_counts(normalize=True).mul(100).round(2))

In [ ]:
# Visualización de la distribución de clases
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de barras
df['target'].value_counts().plot(kind='bar', ax=axes[0], color='steelblue', edgecolor='black')
axes[0].set_title('Distribución de Clases', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Clase', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)

# Añadir valores en las barras
for i, v in enumerate(df['target'].value_counts()):
    axes[0].text(i, v + 100, str(v), ha='center', va='bottom', fontweight='bold')

# Gráfico de pastel
df['target'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', startangle=90)
axes[1].set_title('Proporción de Clases', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

# Análisis de balance de clases
class_counts = df['target'].value_counts()
balance_ratio = class_counts.max() / class_counts.min()
print(f"\nRatio de desbalance: {balance_ratio:.2f}")
if balance_ratio > 3:
    print("⚠️ Dataset DESBALANCEADO - Considerar técnicas de balanceo (SMOTE, pesos de clase)")
else:
    print("✓ Dataset relativamente balanceado")

## 3. Análisis de Valores Faltantes

In [ ]:
# Verificar valores faltantes
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Valores_Faltantes': missing_values,
    'Porcentaje': missing_percent
}).sort_values('Valores_Faltantes', ascending=False)

print("=== ANÁLISIS DE VALORES FALTANTES ===")
if missing_df['Valores_Faltantes'].sum() == 0:
    print("✓ No se encontraron valores faltantes en el dataset")
else:
    print(missing_df[missing_df['Valores_Faltantes'] > 0])

## 4. Estadísticas Descriptivas

In [ ]:
# Separar features y target
feature_cols = [col for col in df.columns if col not in ['id', 'target']]
X = df[feature_cols]
y = df['target']

print(f"Número de características: {len(feature_cols)}")
print(f"\nPrimeras características: {feature_cols[:10]}")

In [ ]:
# Estadísticas descriptivas
X.describe()

In [ ]:
# Distribución de valores por característica
fig, axes = plt.subplots(3, 3, figsize=(18, 12))
axes = axes.ravel()

# Seleccionar 9 características aleatorias para visualizar
np.random.seed(42)
sample_features = np.random.choice(feature_cols, size=min(9, len(feature_cols)), replace=False)

for idx, col in enumerate(sample_features):
    X[col].hist(bins=30, ax=axes[idx], edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribución de {col}', fontsize=10)
    axes[idx].set_xlabel('Valor')
    axes[idx].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

## 5. Análisis de Varianza y Valores Únicos

In [ ]:
# Analizar varianza de las características
variances = X.var().sort_values(ascending=False)

print("=== CARACTERÍSTICAS CON MAYOR VARIANZA ===")
print(variances.head(10))

print("\n=== CARACTERÍSTICAS CON MENOR VARIANZA ===")
print(variances.tail(10))

# Identificar características con varianza muy baja (potencialmente constantes)
low_variance_features = variances[variances < 0.01]
if len(low_variance_features) > 0:
    print(f"\n⚠️ Encontradas {len(low_variance_features)} características con varianza muy baja (< 0.01)")
    print("Considerar eliminarlas durante el preprocesamiento")

In [ ]:
# Visualización de varianzas
plt.figure(figsize=(15, 6))
variances.plot(kind='bar', color='coral', edgecolor='black')
plt.title('Varianza de las Características', fontsize=14, fontweight='bold')
plt.xlabel('Características', fontsize=12)
plt.ylabel('Varianza', fontsize=12)
plt.xticks(rotation=90)
plt.axhline(y=0.01, color='red', linestyle='--', label='Umbral bajo (0.01)')
plt.legend()
plt.tight_layout()
plt.show()

## 6. Análisis de Correlaciones

In [ ]:
# Matriz de correlación (solo para una muestra de características por rendimiento)
# Seleccionar las 20 características con mayor varianza
top_variance_features = variances.head(20).index.tolist()
correlation_matrix = X[top_variance_features].corr()

# Visualizar matriz de correlación
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación (Top 20 características)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Identificar características altamente correlacionadas
def find_high_correlations(corr_matrix, threshold=0.8):
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                high_corr_pairs.append((
                    corr_matrix.columns[i],
                    corr_matrix.columns[j],
                    corr_matrix.iloc[i, j]
                ))
    return high_corr_pairs

# Calcular correlación en todo el dataset
full_corr = X.corr()
high_corr = find_high_correlations(full_corr, threshold=0.8)

if high_corr:
    print("\n=== PARES DE CARACTERÍSTICAS ALTAMENTE CORRELACIONADAS (|r| > 0.8) ===")
    for feat1, feat2, corr_value in high_corr[:10]:  # Mostrar solo las primeras 10
        print(f"{feat1} <-> {feat2}: {corr_value:.3f}")
    print(f"\nTotal de pares: {len(high_corr)}")
else:
    print("✓ No se encontraron pares de características altamente correlacionadas")

## 7. Análisis por Clase

In [ ]:
# Estadísticas por clase
print("=== ESTADÍSTICAS PROMEDIO POR CLASE ===")
print("\nMedia de características por clase (primeras 10 características):")
class_means = df.groupby('target')[feature_cols[:10]].mean()
print(class_means)

In [ ]:
# Comparación de distribuciones entre clases (características con mayor varianza)
top_5_features = variances.head(5).index.tolist()

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, feature in enumerate(top_5_features):
    for target_class in df['target'].unique():
        subset = df[df['target'] == target_class][feature]
        axes[idx].hist(subset, alpha=0.5, label=target_class, bins=20)
    
    axes[idx].set_title(f'Distribución de {feature} por Clase', fontsize=10)
    axes[idx].set_xlabel('Valor')
    axes[idx].set_ylabel('Frecuencia')
    axes[idx].legend()

# Ocultar el subplot vacío
axes[-1].axis('off')

plt.tight_layout()
plt.show()

## 8. Conclusiones del EDA

### Resumen de hallazgos:
1. **Dataset**: Contiene información sobre características numéricas
2. **Clases**: Problema de clasificación multiclase
3. **Balance**: Evaluar si hay desbalance entre clases
4. **Calidad de datos**: Sin valores faltantes
5. **Características**: Múltiples features numéricas que requieren análisis adicional

### Próximos pasos:
- Preprocesamiento de datos (escalado, selección de características)
- Construcción de modelos de clasificación
- Evaluación y comparación de modelos
- Optimización de hiperparámetros

In [ ]:
# Guardar información clave para el siguiente notebook
import json

eda_summary = {
    'n_samples': len(df),
    'n_features': len(feature_cols),
    'n_classes': df['target'].nunique(),
    'class_distribution': df['target'].value_counts().to_dict(),
    'has_missing_values': df.isnull().sum().sum() > 0,
    'low_variance_features': low_variance_features.index.tolist() if len(low_variance_features) > 0 else [],
    'high_corr_pairs_count': len(high_corr)
}

with open('../results/eda_summary.json', 'w') as f:
    json.dump(eda_summary, f, indent=2)

print("✓ Resumen del EDA guardado en 'results/eda_summary.json'")